In [16]:
!pip install transformers datasets torch scikit-learn pandas -q
!pip install pandas -q
!pip install --upgrade transformers torch accelerate datasets scikit-learn pandas -q
!pip list | grep accelerate -q
!pip install accelerate -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

ERROR: Pipe to stdout was broken
Exception ignored in: <_io.TextIOWrapper name='<stdout>' mode='w' encoding='utf-8'>
BrokenPipeError: [Errno 32] Broken pipe


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
import pandas as pd

# Load the dataset
df = pd.read_csv("behavioral_economics_dataset.csv")

# Check dataset structure
print(df.head())

  Student_ID                 Concept  \
0      S1000        Endowment Effect   
1      S1001        Endowment Effect   
2      S1002          Anchoring Bias   
3      S1003  Hyperbolic Discounting   
4      S1004  Hyperbolic Discounting   

                                    Student_Response Faculty_Grade  
0     People fear losses more than they value gains.            B+  
1       Choice architecture can influence decisions.             A  
2  Past investments affect future decisions irrat...             B  
3          Risk perception changes based on framing.            C+  
4       Choice architecture can influence decisions.            C+  


In [18]:
# Rename columns if needed (ensure they match expected format)
df = df.rename(columns={"Student_Response": "text", "Faculty_Grade": "label"})

# Check for missing values
df = df.dropna()


In [19]:
# Convert Grades to Numeric Labels for Classification
grade_mapping = {'A+': 10, 'A': 9, 'A-': 8, 'B+': 7, 'B': 6, 'B-': 5, 'C+': 4, 'C': 3, 'C-': 2, 'D': 1, 'F': 0}
df['label'] = df['label'].map(grade_mapping)

print(df.head()) 

  Student_ID                 Concept  \
0      S1000        Endowment Effect   
1      S1001        Endowment Effect   
2      S1002          Anchoring Bias   
3      S1003  Hyperbolic Discounting   
4      S1004  Hyperbolic Discounting   

                                                text  label  
0     People fear losses more than they value gains.      7  
1       Choice architecture can influence decisions.      9  
2  Past investments affect future decisions irrat...      6  
3          Risk perception changes based on framing.      4  
4       Choice architecture can influence decisions.      4  


In [20]:
from transformers import AutoTokenizer

# Choose a Pretrained Model (e.g., DistilBERT)
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [21]:
# Tokenizing Function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


In [22]:
from datasets import Dataset

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

In [23]:
# Tokenize Data
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map: 100%|███████████████████████████| 100/100 [00:00<00:00, 9970.06 examples/s]


In [24]:
# Split into Train-Test
train_test_split = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

In [25]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch


In [26]:
# Load Pretrained Model for Classification
num_labels = len(grade_mapping)  # Number of grading categories
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
import torch
torch.cuda.empty_cache()

In [30]:
import torch
print(torch.cuda.is_available())  # Should return True if CUDA is available

True


In [31]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    no_cuda=True  # Forces use of CPU even if a GPU is available
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

/home/ashok/langchain/lib/python3.10/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/ashok/langchain/lib/python3.10/site-packages/transformers/training_args.py:1626: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
/tmp/ipykernel_6366/1087228684.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [32]:
# Train Model
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.400387
2,No log,2.398504
3,No log,2.400973


TrainOutput(global_step=30, training_loss=2.3695994059244794, metrics={'train_runtime': 69.8154, 'train_samples_per_second': 3.438, 'train_steps_per_second': 0.43, 'total_flos': 31797278392320.0, 'train_loss': 2.3695994059244794, 'epoch': 3.0})

In [33]:
!pip install numpy

import numpy as np
from sklearn.metrics import accuracy_score, classification_report

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [34]:
# Function to Compute Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}


In [39]:
# Evaluate Model
trainer.evaluate()

model.save_pretrained("/home/ashok/grading_model")
tokenizer.save_pretrained("/home/ashok/grading_model")

model.save_pretrained("/home/ashok/grading_model")
tokenizer.save_pretrained("/home/ashok/grading_model")


('/home/ashok/grading_model/tokenizer_config.json',
 '/home/ashok/grading_model/special_tokens_map.json',
 '/home/ashok/grading_model/vocab.txt',
 '/home/ashok/grading_model/added_tokens.json',
 '/home/ashok/grading_model/tokenizer.json')

In [40]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load trained model and tokenizer
model_path = "/home/ashok/grading_model"  # Use "your-hf-username/grading_model" if using Hugging Face Hub
tokenizer = AutoTokenizer.from_pretrained("/home/ashok/grading_model")
model = AutoModelForSequenceClassification.from_pretrained("/home/ashok/grading_model")


In [41]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Correct grade_mapping used during training
train_grade_mapping = {
    'A+': 10, 'A': 9, 'A-': 8, 'B+': 7, 'B': 6, 'B-': 5, 
    'C+': 4, 'C': 3, 'C-': 2, 'D': 1, 'F': 0
}

# Modify grade mapping for prediction
grade_mapping_for_prediction = {
    10: "A+", 9: "A", 8: "A-", 7: "B+", 6: "B", 5: "B-", 
    4: "C+", 3: "C", 2: "C-", 1: "D", 0: "F"
}

# Function to predict grade
def predict_grade(student_response):
    inputs = tokenizer(student_response, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    
    # Ensure the predicted class is within the valid grade range
    if predicted_class in grade_mapping_for_prediction:
        return grade_mapping_for_prediction[predicted_class]
    else:
        # Handle cases where the predicted class is out of bounds (for safety)
        return "Unknown Grade"

# Example: Predicting grade for a student answer
student_response = "The Industrial Revolution was a period of major industrialization..."
predicted_grade = predict_grade(student_response)
print(f"Predicted Grade: {predicted_grade}")


Predicted Grade: C
